In [2]:
#!pip install tenseal

In [3]:
import tenseal as ts

In [4]:
#the context is required for doing computation on encrypted data , it contains a store for the keys and parameters..
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes =[60,40,40,60])

In [5]:
secret_key = context.secret_key()

In [6]:
import numpy as np 

In [7]:
plain_tensor = np.random.randn(2,3)
plain_tensor

array([[-0.2227758 , -0.14943167, -1.12396358],
       [ 1.11407594,  0.75550481,  0.53377665]])

In [8]:
encrypted_tensor = ts.ckks_tensor(context,plain_tensor,scale=2**40)
encrypted_tensor

In [9]:
# we can set the scale directly to the context that we are not obliged to set it every time we encrypt a tensor 
context.global_scale = 2 ** 40
encrypted_tensor = ts.ckks_tensor(context , plain_tensor)
encrypted_tensor

In [10]:
print(encrypted_tensor.decrypt())

The PlainTensor object is a tensor that we mainly use internally to represent tensors with plain values. You can always call `tolist()` to convert it to a list.

In [11]:
print(encrypted_tensor.decrypt().tolist())

[[-0.22277580317842446, -0.14943167020861553, -1.1239635779208548], [1.114075937780588, 0.7555048142387518, 0.5337766501410985]]


In [12]:
print(encrypted_tensor.decrypt().size())

2


In [13]:
print(plain_tensor)

[[-0.2227758  -0.14943167 -1.12396358]
 [ 1.11407594  0.75550481  0.53377665]]


 # Computation and Evaluation

In [14]:
encrypted_result = (encrypted_tensor + 2) *3 - plain_tensor

In [15]:
expected_result = (plain_tensor +2)*3 - plain_tensor

In [16]:
expected_result

array([[5.5544484 , 5.70113666, 3.75207284],
       [8.22815188, 7.51100962, 7.0675533 ]])

In [17]:
print(encrypted_result.decrypt().tolist())

[[5.554449104650813, 5.701137405838133, 3.7520731955968327], [8.228153126679763, 7.511010740315953, 7.067554320976842]]


In [18]:
vector1 = np.random.randn(3)
vector2 = np.random.randn(3)
enc_vec1 = ts.ckks_tensor(context,vector1)
enc_vec2 = ts.ckks_tensor(context,vector2)
print(f"result : {enc_vec1.dot(enc_vec2).decrypt().tolist()}")
print(f"expected result : {vector1.dot(vector2)}")
print(vector1)
print(vector2)

result : -0.21826350465142294
expected result : -0.21826347941802166
[ 0.45366921 -0.04392263  0.41506866]
[-0.03039485  3.06965999 -0.16779566]


In [19]:
matrix1 = np.random.randn(3,3)
matrix2 = np.random.randn(3,3)
enc_matrix1 = ts.ckks_tensor(context,matrix1)
enc_matrix2 = ts.ckks_tensor(context,matrix2)
print(f"result : \n\t{(enc_matrix1 * enc_matrix2).decrypt().tolist()}")
print(f"\nexpected result : \n\t{matrix1 * matrix2}")

result : 
	[[-1.1261016027490185, -0.01940442610157196, -0.16944022770130393], [-0.08458759294458273, 0.3654973186526257, -0.37304166993013915], [-0.1788734885329819, -0.19370956551046528, -1.9348053085616406]]

expected result : 
	[[-1.12610145 -0.01940442 -0.16944021]
 [-0.08458758  0.36549727 -0.37304162]
 [-0.17887346 -0.19370954 -1.93480505]]


## Batch Computation

In [21]:
# a single ciphertext can hold up to `poly_modulus_degree / 2` values
# so let's use all the slots available
batch_size= 8192 //2
mat1 = np.random.randn(batch_size , 3,3)
mat2 =np.random.randn(3,3)
# batch is by default set to False, we have to turn it on to use the packing feature of ciphertexts
enc_mat1 = ts.ckks_tensor(context,mat1,batch=True)
enc_mat2 = ts.ckks_tensor(context,mat2)
print(f"result : {enc_mat1.dot(enc_mat2).decrypt().tolist()[0]}")
print(f"expected result : {mat1.dot(mat2)[0]}")

#when we st batch=False , it take more time to compute the results then when we enable it 




result : [[-4.367687711694227, -0.832844197917686, -2.7838126028727253], [2.607482391280361, 0.2615827155551822, 1.5119430999202104], [-2.1341662858274355, -0.8209926756804959, -0.9785198944069657]]
expected result : [[-4.36768713 -0.83284408 -2.78381223]
 [ 2.60748205  0.26158268  1.5119429 ]
 [-2.134166   -0.82099257 -0.97851976]]


In [22]:
# TenSEAL use the parelle computation of the computer systeme by default but we can modiying it when creating the context
non_parallel_context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60],
    n_threads=1,
)

In [23]:
# for the Decryption, the context make all the keys private , we can turn it to public but we must save the key for the decryption
sk = context.secret_key()
context.make_context_public()
# by making the context public we dropped the secret key from it, so for the next decryption we need to pass it 

In [24]:
enc_mat1.decrypt()

ValueError: the current context of the tensor doesn't hold a secret_key, please provide one as argument

the Decryption does not work and that because the context does not have a secret key

In [27]:
enc_mat1.decrypt(sk)


You should always make a context public before sending it to other parties to compute on encrypted data.

# Serialization

if we want to send encrypted data or the context we can use the serialize method, every sendable object can be serializable via the serrialize method  

In [28]:
ser_context = context.serialize()
type(ser_context)

bytes

In [34]:
ser_tensor = encrypted_tensor.serialize()
type(ser_tensor)

bytes

there is also a method for deszeialization the context 

In [39]:
loadded_context = ts.context_from(ser_context)
loadded_context 

the tensors must be linked to a context to work properly and this is using the method below

In [42]:
loadded_tensors = ts.ckks_tensor_from(loadded_context,ser_tensor)
loadded_tensors

However, there is also a way to do it the lazy way, deserializing, then linking it to a specific context

In [45]:
lazy_loadded_tensor = ts.lazy_ckks_tensor_from(ser_tensor)
lazy_loadded_tensor + 5

ValueError: missing context

we see that we are not able to manipulate the tensor because it's not linked to a context, so linking the tensor to a context is an obligation for working with the tensors

In [46]:
lazy_loadded_tensor.link_context(loadded_context)


In [47]:
lazy_loadded_tensor + 5